In [2]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta


In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

max_articles = 200
base_url = "https://baomoi.com"
articles = []
page = 1
today = datetime.now()
max_days = timedelta(days=4)

visited_urls = set()

while len(articles) < max_articles:
    url = f"https://baomoi.com/trang{page}.epi"
    print(f"Scraping page: {url}")
    try:
        resp = requests.get(url, timeout=5)
        soup = BeautifulSoup(resp.text, "html.parser")
    except Exception as e:
        print(f"Error fetching page {page}: {e}")
        break

    article_links = soup.select("a[href*='.epi']")
    if not article_links:
        print("No more articles found. Ending.")
        break

    for a in article_links:
        href = a.get("href")
        if not href or not href.endswith(".epi"):
            continue

        article_url = base_url + href
        if article_url in visited_urls:
            continue
        visited_urls.add(article_url)

        try:
            article_resp = requests.get(article_url, timeout=5)
            article_soup = BeautifulSoup(article_resp.text, "html.parser")

            time_tag = article_soup.select_one("time")
            if not time_tag or not time_tag.get("datetime"):
                continue

            pub_time = datetime.strptime(time_tag.get("datetime"), "%Y-%m-%dT%H:%M:%S%z").replace(tzinfo=None)
            if today - pub_time > max_days:
                continue

            article_content = article_soup.get_text(separator=' ', strip=True)
            articles.append(article_content)
            print(f"✅ Collected article {len(articles)} - {article_url}")

        except Exception as e:
            print(f"Error processing article: {e}")
            continue

        if len(articles) >= max_articles:
            break

    page += 1

print(f"\n🎉 Found {len(articles)} articles.")
for i, article in enumerate(articles):
    print(f"\n📰 Article {i + 1}:\n{article[:200]}...\n")


Scraping page: https://baomoi.com/trang1.epi
✅ Collected article 1 - https://baomoi.com/tin-moi.epi
✅ Collected article 2 - https://baomoi.com/tin-video.epi
✅ Collected article 3 - https://baomoi.com/chu-de.epi
✅ Collected article 4 - https://baomoi.com/bong-da.epi
✅ Collected article 5 - https://baomoi.com/tien-ich/ket-qua-xo-so.epi
✅ Collected article 6 - https://baomoi.com/tien-ich/gia-vang.epi
✅ Collected article 7 - https://baomoi.com/tien-ich/thoi-tiet.epi
Error processing article: time data '2025-05-24T06:52:00.000Z' does not match format '%Y-%m-%dT%H:%M:%S%z'
✅ Collected article 8 - https://baomoi.com/bao-dien-tu-tieng-noi-viet-nam-vov-p65.epi
✅ Collected article 9 - https://baomoi.com/tran-duc-luong-t40086853.epi
Error processing article: time data '2025-05-24T08:33:00.000Z' does not match format '%Y-%m-%dT%H:%M:%S%z'
✅ Collected article 10 - https://baomoi.com/bao-vietnamplus-vietnamplus-p293.epi
✅ Collected article 11 - https://baomoi.com/tran-thanh-man-t20431167.epi
Error p